Goal of this program is to run elements of DORA in a way where we can troubleshoot the internal coding pathways

In [ ]:
import sys


### Please append the SMA folder that contains DORA 
sys.path.append(r'D:\Jerry\code\OMMxDORA-tomerge\sma')


import DORA
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import os
import itertools

In [ ]:
#UNIVERSAL PARAMETERS
#get the name of your folder as a string and put r in front 
folder_name = r"D:\Jerry\code\OMMxDORA-tomerge\test_set" #select folder
file_name = 'RecBCD_placeholder_intensity.csv'  
pk = os.path.splitext(file_name)[0]
pixel_size = 117  # in nanometers
time_step = 100  # miliseconds per frame in trajectory movie
frame_start = 0  # enter 0 to start from beginning of dataset #ARBITRARILY SET TO 0 AND -1
frame_end = -1  # enter -1 to end at the last value of the data set
cmap = "spring" # enter a color map string from this https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html
cmappx = "Sunsetdark" # color map string from https://plotly.com/python/builtin-colorscales/
exp_tag = "NaN_debug" # a tag that caries the name of the experiment
first_zero_end = 'no'  # yes to cut off all values after first 0,0 = x,y
graph_centers = "yes" #'yes' or 'no' to graphing the centers of the data. 
save_plot = 'yes' 

exp_title = "_simplest_case"
analysisName='Null'

# file_name = csvnum   #Jerry Gets rid of old file title



#universal parameters
exp_tag = exp_title + pk # a tag that caries the name of the experiment


# pixel_size = 117  # in nanometers
time_step = 2  # miliseconds per frame in trajectory movie
# frame_start = 4000  # enter 0 to start from beginning of dataset
# frame_end = 4500 #
frame_start = 0  # enter 0 to start from beginning of dataset
frame_end = -1
# frame_start = 3500  # enter 0 to start from beginning of dataset
# frame_end = 4000
 # enter -1 to end at the last value of the data set
cmap = "spring_r" # enter a color map string from this https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html
first_zero_end = 'no'  # yes to cut off all values after first 0,0 = x,y
graph_centers = "yes" #'yes' or 'no' to graphing the centers of the data. 
save_plot = 'yes' 

#downsampling parameters
bin_size = 1  # bin size for downsample/filter processing
processing = "none"  # enter downsample, moving average, or none

#Plot Parameters

#Which Graph?
# plot_type = "grid"
plot_type = "angular_continuous"
#Graphing options:
    # Grid plot
        #grid: a grid of little snippets of the data

##### Trajectory Maps Parameters:

# "yes" enables center display of center coordinates if 2D or Find err angle
display_center = "no"

expected_radius = 60 # displays expected ORBIT circular trajectory of given radius (nm)

#Labels
x_axis_label = "x (nm)"
y_axis_label = "y (nm)"
z_axis_label = "Time (ms)"  
unit = "nm"  # enter pixel or nm

#Formatting parameters
pixel_min = -0.75  # setting min/max axis range (pixel)
pixel_max = 0.75
marker_size = 15

# change axis increments for nicely fitting tick marks (pixel)
axis_increment_pixel = 7
# change axis increments for nicely fitting tick marks (nm)
axis_increment_nm = 50
nm_min = -150  # setting min/max axis range (nm)
nm_max = 150
#Do you want to save your plot?
save_plot = 'no'

###Angle Versus Time (AVT or avt) Graphs

#       Formatting parameters for 'radius filter plot'
rad_filter_type_lower = 'nm'  # enter 'zscore' or 'nm' for choice
rad_filter_type_upper = 'zscore'  # enter 'zscore' or 'nm' for choice
z_up = 3  # enter an upper bound for z score.
z_down = -3  # enter a lower bound for z score
dist_low = 30  # lower bound for ABS of Radius filter
dist_high = 70  # upper bound for ABS of Radius filter
filter_nopass = "no" # Filter out non-passing values
annotate_nopass = "yes" #Annotate times of non-passing values

#       Formatting parameters for 'angular_continuous_filtered' plot
# enter 'line' or 'scatter' for a line graph or a scatter plot. Line plot makes up points when hovered
graph_style = 'line'


#Formatting parameters 'grid' plot  ##################################FORMATTING SETTINGS FOR GRID
frames_per_plot = 500  # refers to grid plot
columns = 7  # columns of plots (grid plot)
fig_size_x = 40 # adjust display parameters for graphs to fit nicely, mostly used for 'grid' plot
fig_size_y  = 40 # adjust display parameters for graphs to fit nicely, mostly used for 'grid' plot

#Formatting parameters 'animation' plot
frame_speed = 20  # for animation only (ms)
tail_length = 50  # for animation only

#DORA.table parameters: 
save_table = 'no' #saves filtered table if "yes"

#DORA.Caldera parameters:
donut_width = 15

#DORA.collect_variable parameter package
###DataTable = avt_good cannot have bc it has not been defined yet
col = "Delta Angle"
sample_conditions= analysisName
name_saving_folder = "Data Tables"

### Navigate to the correct folder

In [ ]:
#Change the folder directory to be the current folder's 
os.chdir(folder_name)

#Take all files in the current folder(the one we just switched to) and store it in a list through which we will iterate
my_files = os.listdir(os.getcwd())

# get current path
path_OG = os.getcwd()

In [ ]:
pk = os.path.splitext(file_name)[0]
exp_tag = "Troubleshootings"
#run DORA.find_center
initial_parameters = [file_name, time_step, frame_start, frame_end, cmap, exp_tag, first_zero_end, graph_centers, save_plot]
center, data, ind_invalid_reading, data_back, my_rad_estimate = DORA.find_center(*initial_parameters)

#prompt user for estimated center
# NEW CENTERING ALGORITHM WITH LOW RES HIST

#section x and y data
x = data["X position"]
y = data["Y position"]

# select arbitrary bin value:
bin_num = 50

center_OR = DORA.find_center_hist_max(x, y, bin_num)

In [ ]:
# run DORA.downsampling
processing = "downsample"
downsample_parameters = [bin_size, processing, data, center, time_step, pixel_size, frame_start, frame_end]
if processing == "downsample":
    down_sampled_df,frame_start,frame_end = DORA.downsample(*downsample_parameters)
else:
    down_sampled_df = DORA.downsample(*downsample_parameters)    

### Section to trouble shoot angle calc graphing

In [ ]:
#Graph Groupings:
# create a list of the acceptable groupings for the trajectory maps
trajectory_map = ["2D", "2Dpx", "3D"]

# create a list of the acceptable groupings for the Angle Time grouping
AngleTime = ["radius_filter", "find_err_angle", "angular_continuous",
                "basal3", "angular", "angular_continuous_CR", "find_err_angle_CR"]

# create a list of the acceptable groupings for the Animations Grouping

animations = ["interactive", "animated", "HTML"]


#Trajectory map parameters:
tajectory_map_parameters = [file_name, down_sampled_df, plot_type, display_center, expected_radius, x_axis_label, y_axis_label, z_axis_label, unit, 
pixel_min, pixel_max, axis_increment_nm, axis_increment_pixel, nm_min, nm_max, save_plot, frame_start, frame_end, time_step,cmap,exp_tag,cmappx,marker_size]

#Angle Versus Time (AVT or avt) parameters:
avt_parameters = [file_name, down_sampled_df, plot_type, display_center, ind_invalid_reading, rad_filter_type_upper,
                  rad_filter_type_lower, z_up, z_down, dist_high, dist_low, graph_style, bin_size, frame_start, frame_end,
                  display_center, exp_tag, x_axis_label, y_axis_label, z_axis_label, unit, pixel_min, pixel_max,
                  axis_increment_nm, axis_increment_pixel, nm_min, nm_max, save_plot, data_back, cmap, exp_tag, marker_size, filter_nopass, annotate_nopass] 

#Animated Parameters
animated_parameters = [file_name, down_sampled_df, plot_type, display_center, ind_invalid_reading, rad_filter_type_upper,
                  rad_filter_type_lower, z_up, z_down, dist_high, dist_low, graph_style, bin_size, frame_start, frame_end,
                  display_center, exp_tag, x_axis_label, y_axis_label, z_axis_label, unit, pixel_min, pixel_max,
                  axis_increment_nm, axis_increment_pixel, nm_min, nm_max, save_plot, data_back, cmap, exp_tag, frame_speed, tail_length] 

#Grid Parameters
grid_parameters = [file_name, down_sampled_df, plot_type, display_center, exp_tag, x_axis_label, y_axis_label, z_axis_label, unit, 
pixel_min, pixel_max, axis_increment_nm, axis_increment_pixel, nm_min, nm_max, save_plot, frame_start, frame_end, time_step,cmap,exp_tag, 
frames_per_plot, columns, fig_size_x, fig_size_y]


# #DORA.graph(plot_type,*relevant_parameters)

if plot_type in trajectory_map:
    %matplotlib notebook
    DORA.graph(plot_type,*tajectory_map_parameters)
if plot_type in animations:
    %matplotlib notebook
    DORA.graph(plot_type,*animated_parameters)
if plot_type == "grid":
    DORA.graph(plot_type, *grid_parameters)
if plot_type in AngleTime:
    DORA.graph(plot_type,*avt_parameters)


    


In [ ]:
plot_type

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
 # import the 2 ways to analyze angular data: 1) Claire's way, 2) Jerry's Way [More Current]
import AngleCalc

# Gather inputs to Cacluate Angle under Jerry's Angle calculation paradigm
AngleCalc_inputs = [down_sampled_df, rad_filter_type_upper, rad_filter_type_lower, z_up, z_down, dist_high, dist_low]
# Call and Run Jerry's Angle Calculation
data, data_filtered_pass, data_filtered_nopass, data_filtered_lower_bound_nopass, data_filtered_upper_bound_nopass = AngleCalc.angle_calculations(*AngleCalc_inputs)

In [ ]:
# Data Assignment 
if filter_nopass == "yes":
    df = data_filtered_pass
else:
    df = data
        
# Graphing Section
import plotly.graph_objs as go

# Create a new figure
fig = go.Figure()

# Add the main line trace to the figure
fig.add_trace(go.Scatter(x=df["Time (ms)"], y=df["Continuous Angle"], 
                         name="Raw Data", line = dict(color = 'rgb(36,132,160)')))
       
fig.update_layout(
    xaxis_title="Time (ms)",
    yaxis_title="Continuous Angle (degrees)"
)

# Set plot title
pk = os.path.splitext(file_name)[0]
graph_type = 'Continuous Angle'
list_of_strings = [graph_type, exp_tag]
my_title = "_".join(list_of_strings)
fig.update_layout(title={'text': my_title,'font': {'size': 25},})


# Remove Grid Color and Background color for transparent PNG
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
)

# Update axis line properties
fig.update_layout(
    xaxis=dict(linecolor='black', linewidth=2, mirror=True),
    yaxis=dict(linecolor='black', linewidth=2, mirror=True)
)

# Update x and y axis font 
fig.update_layout(xaxis_title_font=dict(family='Arial', size=22), 
                yaxis_title_font=dict(family='Arial', size=22),
                font_color="black",
                title_font_color="black")

# Remove Gridlines
fig.update_layout(xaxis=dict(showgrid=False, tickfont=dict(size=14)),
                yaxis=dict(showgrid=False, tickfont=dict(size=14)))

if annotate_nopass == "yes":
    # Find times that do not pass the filter and graph a vertical line with different colors
    df_nopass_list = [data_filtered_lower_bound_nopass, data_filtered_upper_bound_nopass, data_back]
    colors = ['rgb(182,0,115)', 'rgb(206,189,222)', 'rgb(55,169,229)'] #pastel red, blue, yellow
    names = ["A", "B", "C"]  # names for each line

    for i, dff in enumerate(df_nopass_list):
        times_nopass = dff["Time (ms)"]
        for t in times_nopass:
            fig.add_vline(x=t, line_width=3, line_dash="dash", line_color=colors[i], name=names[i])
            
    # Show the Legend
    from IPython.display import Image

    # Load a PNG image from a file path
    Image(filename=r'D:\Jerry\code\OMMxDORA-tomerge\Continuous_Angle_Plot_Legend_small.jpg')

        
        

# Show the figure
fig.show()

        